# Import libraries

In [1]:
import os
import pickle
import numpy as np
from tqdm.notebook import tqdm
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Model

## Methods that we used

1. VGG16

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input

2. VGG19

In [ ]:
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input

3. Inceptionv3

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input

4. ResNet50

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input

5. ResNet101

In [ ]:
from tensorflow.keras.applications.resnet import ResNet101, preprocess_input

6. ResNet152

In [ ]:
from tensorflow.keras.applications.resnet import ResNet152, preprocess_input

7. ViT (4 different versions: b16, b32, l16, l32)

In [2]:
from vit_keras import vit

c:\users\admin\appdata\local\programs\python\python38\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [3]:
BASE_DIR = 'flickr30k'    # or change to 'flickr8k'

# Load model without the last layer

CNN example

In [ ]:
# load vgg16 model
model = VGG16()
# restructure the model
model = Model(inputs=model.inputs, outputs=model.layers[-2].output)
# summarize
print(model.summary())

ViT example

In [5]:
image_size = 384

# load vit_b16 model
model = vit.vit_b16(image_size=image_size,
    activation='sigmoid',
    pretrained=True,
    include_top=True,
    pretrained_top=True)
# restructure the model
model = Model(inputs=model.inputs, outputs=model.layers[-2].output)
# summarize
print(model.summary())

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 384, 384, 3)]     0         
                                                                 
 embedding (Conv2D)          (None, 24, 24, 768)       590592    
                                                                 
 reshape_1 (Reshape)         (None, 576, 768)          0         
                                                                 
 class_token (ClassToken)    (None, 577, 768)          768       
                                                                 
 Transformer/posembed_input  (None, 577, 768)          443136    
  (AddPositionEmbs)                                              
                                                                 
 Transformer/encoderblock_0  ((None, 577, 768),        7087872   
  (TransformerBlock)          (None, 12, None, None)       

# Feature extraction

In [ ]:
# extract features from image
features = {}
directory = os.path.join(BASE_DIR, 'images')

for img_name in tqdm(os.listdir(directory)):
    # load the image from file
    img_path = directory + '/' + img_name
    image = load_img(img_path, target_size=(384, 384))    #use (299, 299) for Inceptionv3 and (224, 224) for ResNet and VGG
    # convert image pixels to numpy array
    image = img_to_array(image)
    # reshape data for model
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    # preprocess image
    image = vit.preprocess_inputs(image)    #use image = preprocess_input(image) for CNNs
    # extract features
    feature = model.predict(image, verbose=0)
    # get image ID
    image_id = img_name.split('.')[0]
    # store feature
    features[image_id] = feature


  0%|          | 0/31783 [00:00<?, ?it/s]

In [ ]:
with open('features_flickr30k_vit_b16_384.pickle', 'wb') as handle:
    pickle.dump(features, handle, protocol=pickle.HIGHEST_PROTOCOL)